# 영화추천 기능 구현

## 라이브러리 임포트

In [1]:
import os
from implicit.als import AlternatingLeastSquares
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ['MKL_NUM_THREADS'] = '1'

## 1. 데이터 준비와 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
print("원본:", ratings.shape)
print("결측치 제거 후:", ratings.dropna().shape)

원본: (1000209, 4)
결측치 제거 후: (1000209, 4)


> 결측치는 없다

In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings'] >= 3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size:,}, filtered_data_size: {filtered_data_size:,}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1,000,209, filtered_data_size: 836,478
Ratio of Remaining Data is 83.63%


In [5]:
ratings.rename(columns={'ratings': 'counts'}, inplace=True)
ratings.head(10)

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
print("원본:", movies.shape)
print("결측치 제거 후:", movies.dropna().shape)

원본: (3883, 3)
결측치 제거 후: (3883, 3)


> 결측치는 없다

In [8]:
# 검색 편의를 위해 title, genre 소문자로
movies['title'] = movies['title'].str.lower()
movies['genre'] = movies['genre'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy


In [9]:
# 평점과 제목 데이터프레임 합치기
rating_data = ratings.join(movies.set_index('movie_id'), on='movie_id')
print(rating_data.shape)
rating_data.head()

(836478, 6)


,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),drama
1,1,661,3,978302109,james and the giant peach (1996),animation|children's|musical
2,1,914,3,978301968,my fair lady (1964),musical|romance
3,1,3408,4,978300275,erin brockovich (2000),drama
4,1,2355,5,978824291,"bug's life, a (1998)",animation|children's|comedy


In [10]:
rating_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 1000208
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    836478 non-null  int64 
 1   movie_id   836478 non-null  int64 
 2   counts     836478 non-null  int64 
 3   timestamp  836478 non-null  int64 
 4   title      836478 non-null  object
 5   genre      836478 non-null  object
dtypes: int64(4), object(2)
memory usage: 44.7+ MB


## 2. 분석

* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [11]:
# ratings에 있는 유니크한 영화 개수
print(f"Unique movie_id: {rating_data['movie_id'].nunique():,}")
# ratings에 있는 유니크한 사용자 수
print(f"Unique user_id: {rating_data['user_id'].nunique():,}")

Unique movie_id: 3,628
Unique user_id: 6,039


In [12]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = rating_data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

## 3. 선호하는 영화를 5가지 골라서 ratings에 추가

In [13]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.contains('batman', regex=False)]

,movie_id,title,genre
151,153,batman forever (1995),action|adventure|comedy|crime
588,592,batman (1989),action|adventure|crime|drama
1356,1377,batman returns (1992),action|adventure|comedy|crime
1522,1562,batman & robin (1997),action|adventure|crime
3144,3213,batman: mask of the phantasm (1993),animation|children's


In [14]:
# 나의 아이디는 901203으로 가정, 이미 있는 id인지 체크
rating_data.isin({'user_id':[901203]})['user_id'].any()

False

In [15]:
# 내가 좋아하는 영화 5편 리스트
my_favorite_id = [153, 592, 1377 , 1562, 3213]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# 선정한 5편의 영화를 5번씩 봤다고 가정
my_favorite = pd.DataFrame({'user_id': [901203]*5, 'movie_id': my_favorite_id, 'counts': [5]*5, 'title': my_favorite_title})
my_favorite

,user_id,movie_id,counts,title
0,901203,153,5,batman forever (1995)
1,901203,592,5,batman (1989)
2,901203,1377,5,batman returns (1992)
3,901203,1562,5,batman & robin (1997)
4,901203,3213,5,batman: mask of the phantasm (1993)


In [16]:
# 내 취향 레이팅 데이터에 결합
rating_data = rating_data.append(my_favorite, ignore_index=True)
rating_data.tail()

,user_id,movie_id,counts,timestamp,title,genre
836478,901203,153,5,NaN,batman forever (1995),NaN
836479,901203,592,5,NaN,batman (1989),NaN
836480,901203,1377,5,NaN,batman returns (1992),NaN
836481,901203,1562,5,NaN,batman & robin (1997),NaN
836482,901203,3213,5,NaN,batman: mask of the phantasm (1993),NaN


In [17]:
# 필요 컬럼만 남기고 모두 제거
using_cols = ['user_id', 'movie_id', 'counts', 'title']
rating_data = rating_data[using_cols]
rating_data

,user_id,movie_id,counts,title
0,1,1193,5,one flew over the cuckoo's nest (1975)
1,1,661,3,james and the giant peach (1996)
2,1,914,3,my fair lady (1964)
3,1,3408,4,erin brockovich (2000)
4,1,2355,5,"bug's life, a (1998)"
...,...,...,...,...
836478,901203,153,5,batman forever (1995)
836479,901203,592,5,batman (1989)
836480,901203,1377,5,batman returns (1992)
836481,901203,1562,5,batman & robin (1997)


## 4. CSR matrix를 직접 만들어 봅시다.

In [18]:
# 고유한 유저와 영화의 개수
user_unique = rating_data['user_id'].unique()
movie_unique = rating_data['title'].unique()


# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v: k for k, v in enumerate(user_unique)}
movie_to_idx = {v: k for k, v in enumerate(movie_unique)}

In [19]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = rating_data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(rating_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    rating_data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = rating_data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(rating_data):
    print('title column indexing OK!!')
    rating_data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

rating_data

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,153,5,515
836479,6039,592,5,527
836480,6039,1377,5,587
836481,6039,1562,5,1943


In [20]:
# CSR Matrix 생성
num_user = rating_data['user_id'].nunique()
num_movie = rating_data['title'].nunique()

csr_data = csr_matrix((rating_data['counts'], (rating_data.user_id, rating_data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [21]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [22]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [23]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [24]:
itsme, batman = user_to_idx[901203], movie_to_idx['batman & robin (1997)']
itsme_vector, batman_vector = als_model.user_factors[itsme], als_model.item_factors[batman]

In [25]:
itsme_vector

array([-0.66056   ,  0.4607171 ,  0.6418394 , -0.05152865,  0.09946587,
        0.6482755 , -0.05517736,  1.0396565 , -0.01751541, -0.39804694,
       -0.74770695, -0.27348697,  0.3416256 ,  0.21737161, -0.14270034,
       -0.8306284 ,  0.21972978, -0.05297855,  0.31432146, -0.39885068,
        0.4935285 ,  0.39418438,  0.38306537,  0.2541038 ,  0.2005023 ,
       -1.0647122 ,  0.11739679,  0.02548267,  0.0752346 ,  0.57733613,
       -0.6730967 ,  0.5425104 ,  0.7230321 , -0.7845515 ,  0.20285602,
       -0.13073498,  0.21578734,  0.3895055 , -0.05840174,  0.7332811 ,
        0.12333023, -0.04098547,  0.04770608,  0.11157852,  0.528335  ,
       -0.578999  , -0.3536632 , -0.81755847, -0.10822852, -0.16840537,
        0.00399633,  0.17220552,  0.05376507,  0.5217052 ,  0.9252479 ,
       -0.8950293 ,  0.2630918 ,  0.04213001, -0.00199598, -0.4990407 ,
        0.4234499 ,  0.13828802,  0.48848635, -0.5568425 ,  0.43227285,
        0.46659613,  0.3450636 , -0.44485837, -0.24557666,  0.05

In [26]:
batman_vector

array([ 0.00725098,  0.01991482,  0.01187417,  0.00475931,  0.00880468,
        0.01304824,  0.00050484,  0.00631626, -0.00467162, -0.00165062,
       -0.0038978 , -0.0142313 ,  0.01596441,  0.0070756 ,  0.01416342,
       -0.0006941 ,  0.0228993 ,  0.00373532,  0.0046697 , -0.00481381,
        0.01736959,  0.01205216,  0.00350271,  0.01351355,  0.0031398 ,
       -0.01211409,  0.00799512,  0.00634697,  0.01498764,  0.00797143,
        0.00804115,  0.0093709 ,  0.00251159, -0.01202353,  0.02036694,
        0.00902917,  0.0002889 ,  0.00718609, -0.00026251, -0.00056474,
        0.00432667,  0.00772441,  0.00865959,  0.01749446,  0.01271439,
       -0.00014926,  0.01233794, -0.00598081,  0.00733917, -0.00100375,
        0.01574965,  0.00623341,  0.00891149,  0.02187805,  0.02340629,
       -0.01224949,  0.01077078,  0.0001292 ,  0.00352218, -0.00947324,
        0.01108622,  0.01019233,  0.01769475, -0.00723518,  0.00472549,
        0.025884  ,  0.00753886, -0.01494937, -0.00221305,  0.02

In [27]:
# 나와 스타트렉의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(itsme_vector, batman_vector)

0.34249958

In [28]:
# 다른 영화에 대한 선호도 예측 (선호 리스트에 없는 영화)
matrix = movie_to_idx['bug\'s life, a (1998)']
matrix_vector = als_model.item_factors[matrix]
np.dot(itsme_vector, matrix_vector)

-0.012840501

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [29]:
movie_id = movie_to_idx['batman returns (1992)']
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple

[(587, 1.0),
 (515, 0.855453),
 (781, 0.67831147),
 (1943, 0.64612657),
 (527, 0.5433781),
 (714, 0.50606203),
 (79, 0.5044027),
 (177, 0.50150555),
 (1909, 0.49973345),
 (530, 0.4985712),
 (1085, 0.49703684),
 (766, 0.47372958),
 (497, 0.47189656),
 (1146, 0.44988522),
 (1656, 0.44699106)]

In [30]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['batman returns (1992)',
 'batman forever (1995)',
 'dick tracy (1990)',
 'batman & robin (1997)',
 'batman (1989)',
 'waterworld (1995)',
 'maverick (1994)',
 'three musketeers, the (1993)',
 'addams family values (1993)',
 'lethal weapon 4 (1998)',
 'last action hero (1993)',
 'goldeneye (1995)',
 'addams family, the (1991)',
 'quick and the dead, the (1995)',
 'rumble in the bronx (1995)']

In [31]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [32]:
movies[movies['title'].str.contains('shawshank', regex=False)]

,movie_id,title,genre
315,318,"shawshank redemption, the (1994)",drama


In [33]:
get_similar_movie('shawshank redemption, the (1994)')

['shawshank redemption, the (1994)',
 'silence of the lambs, the (1991)',
 "schindler's list (1993)",
 'pulp fiction (1994)',
 'fargo (1996)',
 'good will hunting (1997)',
 'goodfellas (1990)',
 'saving private ryan (1998)',
 'dead man walking (1995)',
 'usual suspects, the (1995)']

In [34]:
movies[movies['title'].str.contains('superman', regex=False)]

,movie_id,title,genre
2571,2640,superman (1978),action|adventure|sci-fi
2572,2641,superman ii (1980),action|adventure|sci-fi
2573,2642,superman iii (1983),action|adventure|sci-fi
2574,2643,superman iv: the quest for peace (1987),action|adventure|sci-fi


In [35]:
get_similar_movie('superman iii (1983)')

['superman iii (1983)',
 'superman iv: the quest for peace (1987)',
 'howard the duck (1986)',
 'cocoon: the return (1988)',
 'my science project (1985)',
 'superman ii (1980)',
 'karate kid iii, the (1989)',
 'star trek v: the final frontier (1989)',
 'judge dredd (1995)',
 'karate kid, part ii, the (1986)']

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [36]:
user = user_to_idx[901203]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(189, 0.26685297),
 (172, 0.254559),
 (781, 0.25373718),
 (675, 0.25148556),
 (188, 0.22793815),
 (201, 0.21460454),
 (763, 0.20588872),
 (977, 0.1840398),
 (57, 0.18272433),
 (714, 0.175261),
 (790, 0.1662664),
 (154, 0.16559933),
 (702, 0.16222927),
 (372, 0.1599102),
 (58, 0.15795654),
 (526, 0.15787815),
 (253, 0.15784198),
 (80, 0.15508759),
 (52, 0.15505943),
 (177, 0.1549421)]

In [37]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in movie_recommended]

['breakfast club, the (1985)',
 'indiana jones and the last crusade (1989)',
 'dick tracy (1990)',
 'back to the future part iii (1990)',
 'indiana jones and the temple of doom (1984)',
 'who framed roger rabbit? (1988)',
 'superman ii (1980)',
 'adventures of robin hood, the (1938)',
 'untouchables, the (1987)',
 'waterworld (1995)',
 'birdcage, the (1996)',
 'as good as it gets (1997)',
 'field of dreams (1989)',
 'x-men (2000)',
 'mission: impossible (1996)',
 'superman (1978)',
 'mystery men (1999)',
 'stand by me (1986)',
 'dead poets society (1989)',
 'three musketeers, the (1993)']

In [38]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['x-men (2000)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('batman: mask of the phantasm (1993)', 0.061368284321995224),
 ('batman & robin (1997)', 0.04857899070330439),
 ('batman (1989)', 0.0351042920034846),
 ('batman returns (1992)', 0.007366472713301641),
 ('batman forever (1995)', 0.006527238407721786)]

## 정리

* 간단한 영화추천 기능을 구현해봤다. 약식으로 진행했지만 꽤 그럴싸하게 추천해주는 것 같다.
* 배트맨 시리즈를 취향으로 입력했고, 추천해준 결과들은 헐리웃 대작들이다.
* 이전에 회사에서 진행했던 추천서비스는 대학 연구소에서 전달해준 모델을 활용했는데, 직접 작업해보니 흥미로웠다. 앞으로 서비스를 만들 때 참고해야겠다.